In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
data = pd.read_csv('results.csv')

In [3]:
def calculate_statistics(group):
    stats = {}
    stats['mean'] = group['duration'].mean()
    stats['median'] = group['duration'].median()
    stats['std_dev'] = group['duration'].std()
    stats['variance'] = group['duration'].var()

    n = len(group)
    if n > 1:
        se = stats['std_dev'] / np.sqrt(n)
        ci = norm.interval(0.95, loc=stats['mean'], scale=se)
        stats['conf_interval'] = ci
    else:
        stats['conf_interval'] = (stats['mean'], stats['mean'])
    return pd.Series(stats)

In [ ]:
task_stats = data.groupby('task').apply(calculate_statistics)
print(task_stats)

In [ ]:
overall_stats = calculate_statistics(data)
print(overall_stats)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(task_stats.index, task_stats['mean'], color='green', label='Mean Duration')
for i, (mean, ci) in enumerate(zip(task_stats['mean'], task_stats['conf_interval'])):
    ax.errorbar(i + 1, mean, yerr=[[mean - ci[0]], [ci[1] - mean]], fmt='o', color='black', capsize=5)
ax.set_title('Task Performance Analysis')
ax.set_xlabel('Task')
ax.set_ylabel('Duration (seconds)')
ax.set_xticks(task_stats.index)
ax.legend()

plt.tight_layout()
plt.show()